In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb


In [ ]:
path = '~/Studio/seq_bank/data/glu/Glu-CTC-1-1_5p.xlsx'
# path = '~/Studio/seq_bank/data/glu/Glu-CTC-1-1_3p.xlsx'
# path = '~/Studio/seq_bank/data/glu/Glu-CTC-2-1_3p.xlsx'
# path = '~/Studio/seq_bank/data/glu/Glu-TTC-1-1_5p.xlsx'
# path = '~/Studio/seq_bank/data/glu/Glu-TTC-1-1_3p.xlsx'
df_ngs = load_data(path)

glu_ctc_1_1_5p = 'UCCCUGGUGGUCUAGUGGUUAGGAUUCGGCGCUCUCACCGCCGCGGCCCGGGUUCGAUUCCCGGUCAGGGAACCA'
glu_ctc_1_1_3p = glu_ctc_1_1_5p[::-1]

glu_ctc_2_1_5p = 'UCCCUGGUGGUCUAGUGGUUAGGAUUUGGCGCUCUCACCGCCGCGGCCUGGGUUCGAUUCCCGGUCAGGGAACCA'
glu_ctc_2_1_3p = glu_ctc_2_1_5p[::-1]

glu_ttc_1_1_5p = 'UCCCACAUGGUCUAGCGGUUAGGAUUCCUGGUUUUCACCCAGGCGGCCCGGGUUCGACUCCCGGUGUGGGAACCA'
glu_ttc_1_1_3p = glu_ctc_1_1_5p[::-1]

ori, ngs_seq = 5, glu_ctc_1_1_5p
df_ngs['Base'] = list(ngs_seq)
if ori == 5:
    df_ngs['Idx'] = np.array(df_ngs.index)+1
else:
    df_ngs['Idx'] = 75-np.array(df_ngs.index)
df_ngs.head()

In [ ]:
plotly_zone(df_ngs)
df_ngs[['Mass', 'Base', 'Idx']].to_excel('~/Downloads/tmp.xlsx')

In [ ]:
glu_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Glu_FA_17.xlsx'
df_glu1 = load_data(glu_path)
glu_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/18_Glu_A.xlsx'
df_glu1_alkb = load_data(glu_path)

# glu_path = '/Users/xyuan/Documents/ChenLab/8May2021/UC_glu_5_rosa.xlsx'
glu_path = '/Users/xyuan/Documents/ChenLab/DataYue/Glu/DrChen_tRNAs5_complete.xlsx'
df_glu2 = load_data(glu_path)
# glu_path = '/Users/xyuan/Documents/ChenLab/8May2021/UC_glu_6_rosa.xlsx'
glu_path = '/Users/xyuan/Documents/ChenLab/DataYue/Glu/DrChen_tRNAs6_complete.xlsx'
df_glu2_alkb = load_data(glu_path)

df_glu1.shape[0], df_glu1_alkb.shape[0], df_glu2.shape[0], df_glu2_alkb.shape[0]

In [ ]:
plotly_zone(df_glu1_alkb)

In [ ]:
# remove adducts from df_glu1
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_glu1, df_glu1, shift=adduct)
    dfm_adducts_list.append(dfm_r)
df_glu1_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_glu1_without_adducts = df_glu1.drop(df_glu1_adducts.index)
print(df_glu1.shape[0], df_glu1_without_adducts.shape[0], df_glu1_adducts.shape[0])

dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_glu1_alkb, df_glu1_alkb, shift=adduct)
    dfm_adducts_list.append(dfm_r)
df_glu1_alkb_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_glu1_alkb_without_adducts = df_glu1_alkb.drop(df_glu1_alkb_adducts.index)
print(df_glu1_alkb.shape[0], df_glu1_alkb_without_adducts.shape[0], df_glu1_alkb_adducts.shape[0])

In [ ]:
# df_wild, df_alkb = df_glu1.copy(), df_glu1_alkb.copy()
# df_wild_sample = local_top(df_wild, top=20)
# df_alkb_sample = local_top(df_alkb, top=20)

df_wild, df_alkb = df_glu1_without_adducts, df_glu1_alkb_without_adducts
df_wild_sample = df_wild.copy()
df_wild_adducts_sample = df_glu1_adducts.copy()
df_alkb_sample = df_alkb.copy()
df_alkb_adducts_sample = df_glu1_alkb_adducts.copy()
df_wild_sample.shape, df_wild_adducts_sample.shape, df_alkb_sample.shape, df_alkb_adducts_sample.shape

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
# params = [(df_ngs, df_wild_sample, shift) for shift in np.arange(0, 160, 1)]
params = [(df_ngs, df_wild_adducts_sample, shift) for shift in np.arange(0, 160, 1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)

In [ ]:
fig = px.scatter(df_shift_hits[df_shift_hits.hit<df_shift_hits.shape[0]], x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:
df_sh_top = df_shift_hits[df_shift_hits.hit>1]
shifts = df_sh_top['shift']
df_sh_top.shape

In [ ]:
# find out data points which can generate base calling
def func(df_s1, df_s2, shift):
    _, df = peer_dfs(df_s1, df_s2, shift=shift)
    
    bcr = base_calling_random(df, base_only=True)
    df = bcr[0].copy()
    df['Shift'] = shift
    return df

PROCESSES = 8
# params = [(df_ngs, df_wild, shift) for shift in shifts]
params = [(df_ngs, df_wild_adducts_sample, shift) for shift in shifts]
with multiprocessing.Pool(PROCESSES) as pool:
    df_basecallings = pool.starmap(func, params)

df_hits = pd.concat(df_basecallings).drop_duplicates()
df_hits.shape

In [ ]:
# plotly_zones(df_wild_sample, df_hits)
plotly_zone(df_hits, y='Shift')
# plotly_zones(df_hits, dft, y='Shift', names=['All Shifts', 'Selected Shifts'])

In [ ]:
fig = plt.figure(figsize=(10, 6))

df_plt1 = df_hits.copy()

plt.scatter(x=df_plt1['Mass'], y=df_plt1['Shift'])
plt.legend()

fig.tight_layout()
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
plt.savefig(svg_fpath, transparent=True, dpi=300)

In [ ]:
# dft = df_hits[df_hits.Shift.isin([0.0, 14.0, 28.0, 28.1, 42.0, 42.1, 70.0, 70.1, 70.2])]
# target_shifts = [0.0, 14.0, 18, 46]
target_shifts = [0.0, 14.0, 28.0, 28.1, 42.0, 42.1, 56, 70.0, 70.1, 70.2]
dft = df_hits[df_hits.Shift.isin(target_shifts)]
print(dft.shape)
# plotly_zone(dft)
bcr = base_calling_random(dft)
plotly_basecalling(*bcr)

In [ ]:
df_tmp = df_wild_sample[(df_wild_sample.Mass>8701)&(df_wild_sample.Mass<9034)]
plotly_zone(df_tmp, y='Vol')